In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
response.status_code

200

In [3]:
soup = BeautifulSoup(response.text,'html.parser')

d = {'Postal_code':[],'Borough':[]}

table = soup.find_all('table',{'rules':'all'})
for record in table[0].find_all('tr'):
    for i in record.find_all('td'):
        if i.find_all(['a','span'])[0].text == 'Not assigned':
            continue
        d['Postal_code'].append(i.find_all('b')[0].text)
        d['Borough'].append(i.find_all(['a','span'])[0].text)
df = pd.DataFrame(d)
df.head()

,Postal_code,Borough
0,M3A,North York(Parkwoods)
1,M4A,North York(Victoria Village)
2,M5A,Downtown Toronto(Regent Park / Harbourfront)
3,M6A,North York(Lawrence Manor / Lawrence Heights)
4,M7A,Queen's Park(Ontario Provincial Government)


In [4]:
import re
re.search("(?<=\().*(?=\))",'Downtown Toronto(Regent Park / Harbourfront)').group().split('/')[0].strip()

'Regent Park'

In [5]:
df['Neighborhood'] = df.Borough.apply(lambda x: re.search("(?<=\().*(?=\))",x).group().split('/')[0].strip())
df['Borough'] = df.Borough.str.split('(',expand=True)[0]

df['latitude'] = None
df['longitude'] = None

df.head()

,Postal_code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,Regent Park,None,None
3,M6A,North York,Lawrence Manor,None,None
4,M7A,Queen's Park,Ontario Provincial Government,None,None


In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="my-pred")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


for i in range(len(df)):
    address = df.iloc[i,2]+' , Toronto, Ontario'
    location = geocode(address,timeout=3) #timeout=3, dealing with timeout error
    try:
        latitude = location.latitude
        longitude = location.longitude
        df.iloc[i,3] = latitude
        df.iloc[i,4] = longitude
    except:
        print(df.iloc[i,2])
        df.iloc[i,3] = 'NA'
        df.iloc[i,4] = 'NA'

In [ ]:
df = df[df.latitude != 'NA']

df.shape